In [3]:
import pandas as pd
from pathlib import Path
from datetime import datetime 

# Mostra todas as colunas, sem limite
pd.set_option('display.max_columns', None)

# (Opcional) Ajustes que costumam ajudar:
pd.set_option('display.expand_frame_repr', False)   # evita quebra de linha
pd.set_option('display.max_colwidth', None)         # mostra o conteúdo completo das células
pd.set_option('display.width', 0)  

# Pasta com os arquivos Parquet
pasta = Path("/Users/silmara.barnabe/Desktop/Projects/vigimed/data/02_bronze/reacoes")
layer ='03_silver'
dataset = 'reacoes'
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S") #'2025_06_30_14_17_00'
output = "/Users/silmara.barnabe/Desktop/Projects/vigimed/data/"+layer+"/"+dataset+"/"+ts+".parquet"
print(output) 

/Users/silmara.barnabe/Desktop/Projects/vigimed/data/03_silver/reacoes/2025_08_14_11_46_21.parquet


# dates

In [5]:
# Lista todos os arquivos parquet
arquivos = list(pasta.glob("*.parquet"))

dfs = []

for arquivo in arquivos:
    try:
        df_temp = pd.read_parquet(arquivo)
        df_temp['arquivo'] = arquivo.name  # cria coluna com nome do arquivo
        dfs.append(df_temp)
    except Exception as e:
        print(f"Erro ao ler {arquivo.name}: {e}")

# Concatena todos os dataframes lidos
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    print("DataFrames concatenados com sucesso!")
else:
    print("Nenhum arquivo foi carregado.")

# Mostra as primeiras linhas
df_final.head()


DataFrames concatenados com sucesso!


,IDENTIFICACAO_NOTIFICACAO,REACAO_EVTO_ADVERSO_MEDDRA_LLT,PT,HLT,HLGT,SOC,DATA_INICIO_HORA,DATA_FINAL_HORA,DURACAO,GRAVE,GRAVIDADE,DESFECHO,arquivo
0,BR-ANVISA-300000004,Coceira,Prurido,Prurido NCO,Quadros clínicos epidérmicos e dérmicos,Distúrbios dos tecidos cutâneos e subcutâneos,None,None,3 dia,Não,None,Recuperado/Resolvido,2025_08_14_11_22_38.parquet
1,BR-ANVISA-300000005,Edema periorbital,Edema periorbital,Distúrbios oculares NCO,Transtornos oculares NCO,Distúrbios oculares,20181122,20181122,None,Sim,Outro efeito clinicamente significativo,Recuperado/Resolvido,2025_08_14_11_22_38.parquet
2,BR-ANVISA-300000007,Exantema alérgico,Dermatite alérgica,Dermatite e eczema,Quadros clínicos epidérmicos e dérmicos,Distúrbios dos tecidos cutâneos e subcutâneos,20181115,None,2 dia,Sim,Outro efeito clinicamente significativo,Recuperado/Resolvido,2025_08_14_11_22_38.parquet
3,BR-ANVISA-300000008,Flebite,Flebite,Flebite NCO,Infecções e inflamações vasculares,Distúrbios vasculares,20181025,None,5 dia,Sim,Outro efeito clinicamente significativo,Recuperado/Resolvido,2025_08_14_11_22_38.parquet
4,BR-ANVISA-300000010,Parestesia,Parestesia,Parestesias e disestesias,Distúrbios neurológicos NCO,Distúrbios do sistema nervoso,201508,201508,None,Sim,Hospitalização/Prolongamento de hospitalização,Recuperado/Resolvido,2025_08_14_11_22_38.parquet


# Summary

In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125832 entries, 0 to 3125831
Data columns (total 13 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   IDENTIFICACAO_NOTIFICACAO       object
 1   REACAO_EVTO_ADVERSO_MEDDRA_LLT  object
 2   PT                              object
 3   HLT                             object
 4   HLGT                            object
 5   SOC                             object
 6   DATA_INICIO_HORA                object
 7   DATA_FINAL_HORA                 object
 8   DURACAO                         object
 9   GRAVE                           object
 10  GRAVIDADE                       object
 11  DESFECHO                        object
 12  arquivo                         object
dtypes: object(13)
memory usage: 310.0+ MB


In [8]:
def resumo_distintos_colunas(df: pd.DataFrame, cols=None) -> pd.DataFrame:
    """
    Retorna um resumo por coluna com:
    - n_registros (total de linhas)
    - n_distintos
    - %_distintos
    - n_nulos
    - %_nulos
    """
    if cols is None:
        cols = df.columns.tolist()
    df2 = df[cols]
    total = len(df2)

    nunique = df2.nunique(dropna=True)
    nnull = df2.isna().sum()

    out = (
        pd.DataFrame({
            "coluna": nunique.index,
            "n_registros": total,
            "n_distintos": nunique.values,
            "%_distintos": (nunique.values / total * 100) if total else 0.0,
            "n_nulos": nnull.values,
            "%_nulos": (nnull.values / total * 100) if total else 0.0,
        })
        .sort_values("%_distintos", ascending=False)
        .reset_index(drop=True)
    )

    if total:
        out["%_distintos"] = out["%_distintos"].round(2)
        out["%_nulos"] = out["%_nulos"].round(2)

    return out


def resumo_distintos_linhas(df: pd.DataFrame) -> pd.Series:
    """
    Retorna um resumo do dataset considerando todas as colunas:
    - total_linhas
    - linhas_distintas
    - %_linhas_distintas
    - linhas_duplicadas
    - %_linhas_duplicadas
    """
    total = len(df)
    distintos = len(df.drop_duplicates())
    duplicados = total - distintos

    pct_distintos = (distintos / total * 100) if total else 0.0
    pct_duplicados = (duplicados / total * 100) if total else 0.0

    return pd.Series({
        "total_linhas": total,
        "linhas_distintas": distintos,
        "%_linhas_distintas": round(pct_distintos, 2),
        "linhas_duplicadas": duplicados,
        "%_linhas_duplicadas": round(pct_duplicados, 2),
    })



In [11]:

# --- USO ---
cols = [
    "IDENTIFICACAO_NOTIFICACAO","REACAO_EVTO_ADVERSO_MEDDRA_LLT","PT","HLT","HLGT",
    "SOC","DATA_INICIO_HORA","DATA_FINAL_HORA","DURACAO","GRAVE","GRAVIDADE","DESFECHO","arquivo"
]

summary_cols = resumo_distintos_colunas(df_final, cols=cols)
summary_rows = resumo_distintos_linhas(df_final)
 

In [12]:
summary_cols

,coluna,n_registros,n_distintos,%_distintos,n_nulos,%_nulos
0,IDENTIFICACAO_NOTIFICACAO,3125832,299608,9.58,0,0.00
1,DATA_INICIO_HORA,3125832,25134,0.80,1378420,44.10
2,DATA_FINAL_HORA,3125832,19198,0.61,2243796,71.78
3,REACAO_EVTO_ADVERSO_MEDDRA_LLT,3125832,17878,0.57,62528,2.00
4,PT,3125832,8867,0.28,62528,2.00
5,HLT,3125832,1339,0.04,62528,2.00
6,DURACAO,3125832,995,0.03,2642784,84.55
7,HLGT,3125832,321,0.01,62528,2.00
8,GRAVIDADE,3125832,56,0.00,2100472,67.20
9,SOC,3125832,27,0.00,62528,2.00


In [13]:
print("\nResumo por linhas do dataset:\n", summary_rows)


Resumo por linhas do dataset:
 total_linhas           3125832.00
linhas_distintas        776298.00
%_linhas_distintas          24.83
linhas_duplicadas      2349534.00
%_linhas_duplicadas         75.17
dtype: float64


# DATA_INICIO_HORA

In [14]:
df_final.DATA_INICIO_HORA.value_counts().sort_index(ascending=False).head(10)

DATA_INICIO_HORA
241213               4
241209               4
241127               4
241010               4
230806               4
22:18:00             4
22:15:00             4
2030826              4
20290611 23:00       4
20251220 21:40:00    4
Name: count, dtype: int64

In [15]:
df_final.DATA_FINAL_HORA.value_counts().sort_index(ascending=False).head(10)

DATA_FINAL_HORA
4                    4
241202               4
23:10:00             4
22:00:00             8
2240219              4
21:30                4
20:00:00             8
20420809             4
20260529             4
20251220 22:00:00    4
Name: count, dtype: int64

# GRAVE

In [19]:
df_final.GRAVE.value_counts().sort_index(ascending=True).head(10)

GRAVE
Não    1429552
Sim    1026520
Name: count, dtype: int64

# DESFECHO

In [18]:
df_final.DESFECHO.value_counts().sort_index(ascending=False)

DESFECHO
Recuperado/Resolvido com sequelas              16532
Recuperado/Resolvido                         1061092
Não Recuperado/Não Resolvido/Em andamento     327380
Fatal/Óbito                                    71504
Em recuperação/Resolvendo                     255600
Desconhecido                                  867860
Name: count, dtype: int64

# SOC

In [20]:
df_final.SOC.value_counts().sort_index(ascending=False)

SOC
Quadros clínicos na gravidez, no puerpério e perinatais                        10352
Procedimentos cirúrgicos e médicos                                             36076
Problemas relacionados ao produto                                              28016
Neoplasias benignas, malignas e não especificadas (incl. cistos e pólipos)     43532
Lesões, intoxicações e complicações de procedimentos                          321104
Investigações                                                                 111568
Infecções e infestações                                                       122952
Distúrbios vasculares                                                         138688
Distúrbios respiratórios, torácicos e do mediastino                           177264
Distúrbios renais e urinários                                                  30072
Distúrbios psiquiátricos                                                       94524
Distúrbios oculares                                          

# GRAVIDADE

In [21]:
df_final.GRAVIDADE.value_counts().sort_index(ascending=False)

GRAVIDADE
Resultou em óbito, Outro efeito clinicamente significativo                                                                                                                                                            12032
Resultou em óbito, Incapacidade persistente ou significativa, Outro efeito clinicamente significativo                                                                                                                    40
Resultou em óbito, Incapacidade persistente ou significativa, Hospitalização/Prolongamento de hospitalização, Outro efeito clinicamente significativo                                                                    32
Resultou em óbito, Incapacidade persistente ou significativa, Hospitalização/Prolongamento de hospitalização                                                                                                            280
Resultou em óbito, Incapacidade persistente ou significativa, Ameaça à vida, Outro efeito clinicamente signifi